In [1]:
import numpy as np
from typing import List, Tuple
from skimage.io import imread
from skimage.color import gray2rgb
from skimage.transform import resize


class ImageVectorizer:
    def __init__(self, dims: Tuple[int, int]):
        self.dims = dims
        self.num_channels = 3

    def convert_image_to_array(self, image_path: str) -> np.ndarray:
        image_array = imread(image_path)
        if len(image_array.shape) == 2:  # grayscale image
            image_array = gray2rgb(image_array)
        elif image_array.shape[2] > self.num_channels:
            image_array = image_array[:, :, : self.num_channels]
        image_array = resize(
            image_array, (self.dims[0], self.dims[1]),
            anti_aliasing=True
        )
        return image_array

    def extract_features_from_images(
        self, image_paths: List[str]
    ) -> np.ndarray:
        num_images = len(image_paths)
        image_features = np.zeros(
            (num_images, self.dims[0], self.dims[1], self.num_channels),
            dtype=np.float32
        )
        for idx, img_path in enumerate(image_paths):
            image_features[idx] = self.convert_image_to_array(img_path)
        return image_features


In [2]:
import os
PROJECT_DIR = "/Users/abraham/Documents/Work/Self/2024/sentiment_ai"
datasets_dir = os.path.join(PROJECT_DIR, "datasets")

In [3]:
from glob import glob
fer_train_dir = os.path.join(datasets_dir, 'FER2013', 'train')
fer_test_dir = os.path.join(datasets_dir, 'FER2013', 'test')

def collect_labeled_images_from_dir(dir_path):
    # Directory_sentiment map
    dir_sentiment_map = {
        'happy': 0,
        'sad': 1,
        'fear': 2,
        'surprise': 3,
        'neutral': 4,
        'angry': 5,
        'disgust': 6,
    }
    labeled_images = {}
    for sentiment, label in dir_sentiment_map.items():
        sentiment_img_paths = os.path.join(dir_path, sentiment)
        for img_path in glob(os.path.join(sentiment_img_paths, '*jpg')):
            labeled_images[img_path] = label
    return labeled_images

In [4]:
from typing import Dict, Tuple
dims = (256, 256)
img_vectortizer = ImageVectorizer(dims=dims)

def extract_features_and_labels(
    labeled_images: Dict[str, int], vectorizer: ImageVectorizer
)-> Tuple[np.ndarray, np.ndarray]:
    image_paths = list(labeled_images.keys())
    labels = list(labeled_images.values())
    # Extract features
    image_features = vectorizer.extract_features_from_images(image_paths)
    one_hot_labels = utils.to_categorical(labels, num_classes=len(set(labels)))
    return image_features, one_hot_labels


In [5]:
from keras import utils
dims = (256, 256)
img_vectorizer = ImageVectorizer(dims=dims)
train_images = collect_labeled_images_from_dir(fer_train_dir)
X_train, y_train = extract_features_and_labels(
    train_images, img_vectorizer)

In [6]:
test_images = collect_labeled_images_from_dir(fer_test_dir)
X_test, y_test = extract_features_and_labels(test_images, img_vectorizer)

In [7]:
X_train.shape

(28709, 256, 256, 3)

In [8]:
y_train.shape

(28709, 7)

In [9]:
from keras import layers, models, optimizers, callbacks

class FaceSentimentCNNModel:

    def __init__(self, image_shape, num_sentiments):
        self.image_shape = image_shape
        self.num_sentiments = num_sentiments
        self.model = self._build_model()
        self._compile_model()
    
    def _build_model(self):
        model = models.Sequential([
            layers.Input(shape=self.image_shape),
            
            layers.Conv2D(
                32, kernel_size=(3, 3), padding='same', 
                kernel_initializer='glorot_uniform', bias_initializer='zeros'
            ),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            layers.Dropout(0.25),
            
            layers.Conv2D(
                64, kernel_size=(3, 3), padding='same', 
                kernel_initializer='glorot_uniform', bias_initializer='zeros'
            ),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            layers.Dropout(0.25),
            
            layers.Conv2D(
                128, kernel_size=(3, 3), padding='same', 
                kernel_initializer='glorot_uniform', bias_initializer='zeros'
            ),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            layers.Dropout(0.25),
            
            layers.Conv2D(
                256, kernel_size=(3, 3), padding='same', 
                kernel_initializer='glorot_uniform', bias_initializer='zeros'
            ),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            
            layers.Conv2D(
                256, kernel_size=(3, 3), padding='same', 
                kernel_initializer='glorot_uniform', bias_initializer='zeros'
            ),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.AveragePooling2D(pool_size=(2, 2)),
            layers.Dropout(0.5),
            
            layers.Flatten(),
            
            layers.Dense(
                512, kernel_initializer='glorot_uniform', 
                bias_initializer='zeros'
            ),
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Dropout(0.5),
            
            layers.Dense(
                self.num_sentiments, kernel_initializer='glorot_uniform', 
                bias_initializer='zeros'
            ),
            layers.Activation('softmax')
        ])
        return model

    def _compile_model(self):
            fer_optimizer = optimizers.Adam(learning_rate=0.001)
            self.model.compile(
                optimizer=fer_optimizer, loss='categorical_crossentropy', 
                metrics=['accuracy']
            )
    
    def get_model(self):
        return self.model


In [10]:
image_shape = (dims[0], dims[1], 3)
batch_size = 32
epochs = 30
num_sentiments=7
cnn_model = FaceSentimentCNNModel(image_shape, num_sentiments)
model = cnn_model.get_model()
checkpoint_dir = os.path.join(PROJECT_DIR, 'models')
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint = callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'face_modelCNN.keras'),
    monitor='val_loss', save_best_only=True,
    verbose=1
)

In [ ]:
model.fit(
    X_train, y_train, batch_size=batch_size, epochs=epochs, 
    validation_data=(X_test, y_test), shuffle=True, callbacks=[checkpoint]
)

Epoch 1/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2706 - loss: 2.0134
Epoch 1: val_loss improved from inf to 1.90018, saving model to /Users/abraham/Documents/Work/Self/2024/sentiment_ai/models/face_modelCNN.keras
898/898 ━━━━━━━━━━━━━━━━━━━━ 1427s 2s/step - accuracy: 0.2706 - loss: 2.0132 - val_accuracy: 0.2106 - val_loss: 1.9002
Epoch 2/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4279 - loss: 1.4889
Epoch 2: val_loss improved from 1.90018 to 1.40048, saving model to /Users/abraham/Documents/Work/Self/2024/sentiment_ai/models/face_modelCNN.keras
898/898 ━━━━━━━━━━━━━━━━━━━━ 1395s 2s/step - accuracy: 0.4279 - loss: 1.4888 - val_accuracy: 0.4570 - val_loss: 1.4005
Epoch 3/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5082 - loss: 1.2918
Epoch 3: val_loss did not improve from 1.40048
898/898 ━━━━━━━━━━━━━━━━━━━━ 1381s 2s/step - accuracy: 0.5082 - loss: 1.2918 - val_accuracy: 0.4210 - val_loss: 1.5630
Epoch 4/30
898/898 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/ste